In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [3]:
from google.colab import files
uploaded = files.upload()


Saving archive (8).zip to archive (8).zip


In [6]:
import zipfile
import os

# Define file paths
zip_file_path = "archive (8).zip"
extract_folder = "extracted_data"

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# List extracted files
os.listdir(extract_folder)


['Retail-Supply-Chain-Sales-Dataset.xlsx']

In [7]:
import pandas as pd

# Identify the extracted Excel file (Replace with actual filename)
file_path = os.path.join(extract_folder, "Retail-Supply-Chain-Sales-Dataset.xlsx")

# Load the Excel file and check sheet names
xls = pd.ExcelFile(file_path)
print("Available Sheets:", xls.sheet_names)

# Load data
retail_data = pd.read_excel(xls, sheet_name="Retails Order Full Dataset")
calendar_data = pd.read_excel(xls, sheet_name="Calendar Table")

# Display first few rows
retail_data.head()


Available Sheets: ['Retails Order Full Dataset', 'Calendar Table']


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Retail Sales People,Product ID,Category,Sub-Category,Product Name,Returned,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Cassandra Brandow,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,Not,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Cassandra Brandow,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Not,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-12-06,2016-12-06,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Anna Andreadi,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,Not,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-11-10,2015-11-10,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Cassandra Brandow,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,Not,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-11-10,2015-11-10,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Cassandra Brandow,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,Not,22.3680,2,0.20,2.5164


In [34]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Row ID               9994 non-null   int64         
 1   Order ID             9994 non-null   object        
 2   Order Date           9994 non-null   datetime64[ns]
 3   Ship Date            9994 non-null   datetime64[ns]
 4   Ship Mode            9994 non-null   object        
 5   Customer ID          9994 non-null   object        
 6   Customer Name        9994 non-null   object        
 7   Segment              9994 non-null   object        
 8   Country              9994 non-null   object        
 9   City                 9994 non-null   object        
 10  State                9994 non-null   object        
 11  Postal Code          9994 non-null   int64         
 12  Region               9994 non-null   object        
 13  Retail Sales People  9994 non-nul

In [9]:
# Conducting the Analysis

# (A) Retail Sales Performance & Profitability

In [10]:
sales_summary = retail_data.agg({
    'Sales': ['sum', 'mean', 'median', 'min', 'max'],
    'Profit': ['sum', 'mean', 'median', 'min', 'max'],
    'Quantity': ['sum', 'mean', 'median', 'min', 'max']
})

print("Sales & Profitability Summary")
print(sales_summary)


Sales & Profitability Summary
               Sales         Profit      Quantity
sum     2.297201e+06  286397.021700  37873.000000
mean    2.298580e+02      28.656896      3.789574
median  5.449000e+01       8.666500      3.000000
min     4.440000e-01   -6599.978000      1.000000
max     2.263848e+04    8399.976000     14.000000


In [11]:
# (B) Customer Trends & Product Preferences

In [14]:
top_selling_products = retail_data.groupby('Product Name').agg({'Sales': 'sum', 'Quantity': 'sum'}).sort_values(by='Sales', ascending=False).head(10)
#Formatting sales values with a dollar sign
top_selling_products['Sales'] = top_selling_products['Sales'].apply(lambda x: f"${x:,.2f}")


print("Top Selling Products")
print(top_selling_products)




Top Selling Products
                                                         Sales  Quantity
Product Name                                                            
Canon imageCLASS 2200 Advanced Copier               $61,599.82        20
Fellowes PB500 Electric Punch Plastic Comb Bind...  $27,453.38        31
Cisco TelePresence System EX90 Videoconferencin...  $22,638.48         6
HON 5400 Series Task Chairs for Big and Tall        $21,870.58        39
GBC DocuBind TL300 Electric Binding System          $19,823.48        37
GBC Ibimaster 500 Manual ProClick Binding System    $19,024.50        48
Hewlett Packard LaserJet 3310 Copier                $18,839.69        38
HP Designjet T520 Inkjet Large Format Printer -...  $18,374.90        12
GBC DocuBind P400 Electric Binding System           $17,965.07        27
High Speed Automatic Electric Letter Opener         $17,030.31        11


In [15]:
top_profitable_products = retail_data.groupby('Product Name').agg({'Profit': 'sum'}).sort_values(by='Profit', ascending=False).head(10)
# Formatting Profit Values in a dollar sign
top_profitable_products['Profit'] = top_profitable_products['Profit'].apply(lambda x: f"${x:,.2f}")
print("Top Profitable Products")
print(top_profitable_products)



Top Profitable Products
                                                        Profit
Product Name                                                  
Canon imageCLASS 2200 Advanced Copier               $25,199.93
Fellowes PB500 Electric Punch Plastic Comb Bind...   $7,753.04
Hewlett Packard LaserJet 3310 Copier                 $6,983.88
Canon PC1060 Personal Laser Copier                   $4,570.93
HP Designjet T520 Inkjet Large Format Printer -...   $4,094.98
Ativa V4110MDD Micro-Cut Shredder                    $3,772.95
3D Systems Cube Printer, 2nd Generation, Magenta     $3,717.97
Plantronics Savi W720 Multi-Device Wireless Hea...   $3,696.28
Ibico EPK-21 Electric Binding System                 $3,345.28
Zebra ZM400 Thermal Label Printer                    $3,343.54


In [17]:
top_customer_segments = retail_data.groupby('Segment').agg({'Sales': 'sum', 'Profit': 'sum'}).sort_values(by='Sales', ascending=False)
top_customer_segments['Profit'] = top_customer_segments['Profit'].apply(lambda x: f"${x:,.2f}")
top_customer_segments['Sales'] = top_customer_segments['Sales'].apply(lambda x: f"${x:,.2f}")

print("Top Customer Segments")
print(top_customer_segments)

Top Customer Segments
                     Sales       Profit
Segment                                
Consumer     $1,161,401.34  $134,119.21
Corporate      $706,146.37   $91,979.13
Home Office    $429,653.15   $60,298.68


In [19]:
top_customers = retail_data.groupby('Customer Name').agg({'Sales': 'sum', 'Order ID': 'count'}).sort_values(by='Sales', ascending=False).head(10)
top_customers['Sales'] = top_customers['Sales'].apply(lambda x: f"${x:,.2f}")
print("Top Customers")
print(top_customers)

Top Customers
                         Sales  Order ID
Customer Name                           
Sean Miller         $25,043.05        15
Tamara Chand        $19,052.22        12
Raymond Buch        $15,117.34        18
Tom Ashbrook        $14,595.62        10
Adrian Barton       $14,473.57        20
Ken Lonsdale        $14,175.23        29
Sanjit Chand        $14,142.33        22
Hunter Lopez        $12,873.30        11
Sanjit Engle        $12,209.44        19
Christopher Conant  $12,129.07        11


In [20]:
# (C) Geographic & Temporal Analysis

In [22]:
# Sales by Country
sales_by_country = retail_data.groupby('Country').agg({'Sales': 'sum', 'Profit': 'sum'}).sort_values(by='Sales', ascending=False)
sales_by_country['Profit'] = sales_by_country['Profit'].apply(lambda x: f"${x:,.2f}")
sales_by_country['Sales'] = sales_by_country['Sales'].apply(lambda x: f"${x:,.2f}")

print("Sales by Country")
print(sales_by_country)

Sales by Country
                       Sales       Profit
Country                                  
United States  $2,297,200.86  $286,397.02


In [23]:
# Sales by City (Top 10)
sales_by_city = retail_data.groupby('City').agg({'Sales': 'sum', 'Profit': 'sum'}).sort_values(by='Sales', ascending=False).head(10)
sales_by_city['Profit'] = sales_by_city['Profit'].apply(lambda x: f"${x:,.2f}")
sales_by_city['Sales'] = sales_by_city['Sales'].apply(lambda x: f"${x:,.2f}")

print("Sales by City")
print(sales_by_city)


Sales by City
                     Sales       Profit
City                                   
New York City  $256,368.16   $62,036.98
Los Angeles    $175,851.34   $30,440.76
Seattle        $119,540.74   $29,156.10
San Francisco  $112,669.09   $17,507.39
Philadelphia   $109,077.01  $-13,837.77
Houston         $64,504.76  $-10,153.55
Chicago         $48,539.54   $-6,654.57
San Diego       $47,521.03    $6,377.20
Jacksonville    $44,713.18   $-2,323.84
Springfield     $43,054.34    $6,200.70


In [25]:
# Sales Over Time (Monthly)
retail_data['Order Date'] = pd.to_datetime(retail_data['Order Date'])
sales_by_month = retail_data.resample('M', on='Order Date').agg({'Sales': 'sum', 'Profit': 'sum'})
sales_by_month['Profit'] = sales_by_month['Profit'].apply(lambda x: f"${x:,.2f}")
sales_by_month['Sales'] = sales_by_month['Sales'].apply(lambda x: f"${x:,.2f}")
print("Sales Over Time")
print(sales_by_month)

Sales Over Time
                 Sales      Profit
Order Date                        
2014-01-31  $28,953.71   $4,549.45
2014-02-28  $12,743.11   $2,654.56
2014-03-31  $54,801.91      $92.70
2014-04-30  $24,710.02   $4,601.07
2014-05-31  $29,639.83   $3,912.25
2014-06-30  $29,287.03   $4,499.74
2014-07-31  $35,341.25  $-1,783.54
2014-08-31  $37,854.55   $2,081.27
2014-09-30  $66,110.22  $10,232.26
2014-10-31  $34,561.95   $4,075.15
2014-11-30  $64,817.62   $6,653.28
2014-12-31  $65,426.31   $7,975.78
2015-01-31  $29,347.39   $1,237.06
2015-02-28  $20,728.35   $2,171.40
2015-03-31  $40,876.61   $9,165.68
2015-04-30  $38,056.97   $6,132.88
2015-05-31  $30,933.71   $2,779.40
2015-06-30  $28,862.20   $4,788.45
2015-07-31  $28,730.38     $671.80
2015-08-31  $50,094.53  $10,112.28
2015-09-30  $66,729.33   $9,929.30
2015-10-31  $32,025.08   $3,433.24
2015-11-30  $50,732.31   $5,582.40
2015-12-31  $53,415.66   $5,614.71
2016-01-31  $38,048.18   $8,501.17
2016-02-29  $49,238.41  $14,683.72
2016

<ipython-input-25-1d7189b288b4>:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sales_by_month = retail_data.resample('M', on='Order Date').agg({'Sales': 'sum', 'Profit': 'sum'})


In [26]:
# (D) Demand Forecasting Model


In [27]:
# Extract date-based features
retail_data['Year'] = retail_data['Order Date'].dt.year
retail_data['Month'] = retail_data['Order Date'].dt.month

In [28]:
# Aggregate demand by month
demand_data = retail_data.groupby(['Year', 'Month']).agg({'Quantity': 'sum'}).reset_index()

In [29]:
# Train-test split
X = demand_data[['Year', 'Month']]
y = demand_data['Quantity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
# Train RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [31]:
# Predictions
y_pred = model.predict(X_test)

In [32]:
# Model Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

In [33]:
print("Demand Forecasting Model Evaluation:")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Demand Forecasting Model Evaluation:
Mean Absolute Error: 160.35299999999998
Mean Squared Error: 44503.68457
Root Mean Squared Error: 210.95896418498077
